In [4]:
import os
import os.path as p
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AdamW, AutoTokenizer, AutoModel

from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook

import warnings
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [23]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [24]:
train_xnli.head()

,sentence1,sentence2,gold_label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment
2,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,entailment
3,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,entailment
4,"그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...",테니스화의 가격은 다양하다.,neutral


In [25]:
#결합 후 섞기
train_data = train_snli.append(train_xnli)
train_data = train_data.sample(frac=1)

In [26]:
train_data.head()

,sentence1,sentence2,gold_label
158515,성과가 높은 조직은 또한 조직의 낮은 수준으로 권한을 위임함으로써 직원을 참여시키고...,실적이 낮은 조직들도 직원들을 참여시키고 참여시키기 위해 노력한다,neutral
217758,"하나는 다른 하나가 빼앗은 것을 복원한다.""",하나는 다른 하나를 대신한다.,neutral
453696,헤드기어와 복싱 글러브를 착용한 남성이 복싱 글러브만 착용한 여성과 함께 훈련하고 있다.,한 사람이 장갑을 낀 여자와 훈련을 하고 있다.,entailment
124475,아름다운 날에 한 남자가 골프를 치고 있다.,테니스 치는 여자.,contradiction
115197,"셔츠, 바지, 모자를 매치한 세 남자와 가운을 입은 한 남자가 방에 있다.",방에 있는 남자들,entailment


In [27]:
def drop_na_and_duplicates(df) :
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

In [28]:
#결측값 및 중복 샘플 제거
train_data = drop_na_and_duplicates(train_data)
val_data = drop_na_and_duplicates(val_data)
test_data = drop_na_and_duplicates(test_data)

In [29]:
train_data

,sentence1,sentence2,gold_label
0,성과가 높은 조직은 또한 조직의 낮은 수준으로 권한을 위임함으로써 직원을 참여시키고...,실적이 낮은 조직들도 직원들을 참여시키고 참여시키기 위해 노력한다,neutral
1,"하나는 다른 하나가 빼앗은 것을 복원한다.""",하나는 다른 하나를 대신한다.,neutral
2,헤드기어와 복싱 글러브를 착용한 남성이 복싱 글러브만 착용한 여성과 함께 훈련하고 있다.,한 사람이 장갑을 낀 여자와 훈련을 하고 있다.,entailment
3,아름다운 날에 한 남자가 골프를 치고 있다.,테니스 치는 여자.,contradiction
4,"셔츠, 바지, 모자를 매치한 세 남자와 가운을 입은 한 남자가 방에 있다.",방에 있는 남자들,entailment
...,...,...,...
941809,중국 시장에서 카메라를 보고 있는 관광객 두어 명.,관광객들이 호텔에 있다.,contradiction
941810,판사가 배심원을 상대로 한 판결에 대한 흥미로운 어 포인트 나는 그것과 함께 올 수...,판사의 판결에 대한 흥미로운 점 나는 그것과 함께 올 학대를 받아들일지 모르겠다.,entailment
941811,enclos paroissial(파리쉬 클로즈)은 16세기에서 18세기 동안 브리트...,가까운 교구는 현대적인 도시 환경이다.,contradiction
941812,"알고 보니, 적어도 우리만큼 따뜻하지는 않지만, 그들을 우리의 직장과 학교에 환영하...",우리가 전에 그랬던 것처럼 그들을 환영해도 괜찮다.,contradiction


In [7]:
#kobert토크나이저 pre train 버트모델 불러와 사용
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import gluonnlp as nlp
import numpy as np

In [8]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [9]:
class koBERTNLI(nn.Module):
    def __init__(self, kobert, hidden_size=768):
        super().__init__()
        self.kobert = kobert
        self.classifier = nn.Linear(hidden_size, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.kobert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        outputs = self.classifier(outputs[1])
        outputs = torch.nn.functional.softmax(outputs, dim=1)
        return outputs

In [10]:
label_to_num_dict = {
    'contradiction':0,
    'entailment':1,
    'neutral':2
}

In [11]:
#두 문장을 입력받아서 하나의 input으로 바꿔줌
class NLIDataset(Dataset):
  def __init__(self, dataset, tokenizer):
    self.tokenizer = tokenizer
    self.dataset = dataset
    sentence1 = self.dataset['sentence1'].tolist()
    sentence2 = self.dataset['sentence2'].tolist()
    inputs = tokenizer(sentence1, sentence2, padding=True, truncation=True)
    self.input_ids = inputs['input_ids']
    self.attention_mask = inputs['attention_mask']
    self.token_type_ids = inputs['token_type_ids']

    self.labels = self.dataset['gold_label'].tolist()
    self.labels = [label_to_num_dict[l] for l in self.labels]
    self.labels = F.one_hot(torch.tensor(self.labels), num_classes=3)
    self.labels = self.labels.tolist()

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return (self.input_ids[idx], self.attention_mask[idx], self.token_type_ids[idx], self.labels[idx])

In [12]:
def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    attention_mask = [item[1] for item in batch]
    token_type_ids = [item[2] for item in batch]
    labels = [item[3] for item in batch]
    return torch.LongTensor(input_ids), torch.LongTensor(attention_mask), torch.LongTensor(token_type_ids), torch.FloatTensor(labels)

In [30]:
train_ds = NLIDataset(train_data, tokenizer)
valid_ds = NLIDataset(val_data, tokenizer)
test_ds = NLIDataset(test_data, tokenizer)

In [31]:
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
model = koBERTNLI(kobert=bertmodel)

cuda:0


In [32]:
model = model.to(device)

In [52]:
#하이퍼파라미터
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
log_interval = 1000
learning_rate = 5e-5

In [47]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn)
valid_dataloader = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size, collate_fn=collate_fn)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, collate_fn=collate_fn)

In [48]:
total_steps = len(train_dataloader) * num_epochs
warmup_step = int(total_steps * warmup_ratio)

In [49]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_step,
                                            num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss()

In [51]:
#실제 학습 진행 부분
best_loss = float('inf')
print(device)
for e in range(num_epochs):
    epoch_loss = 0.0
    valid_loss = 0.0
    model.train()
    for batch_id, (input_ids, attention_mask, token_type_ids, labels) in enumerate(tqdm_notebook(train_dataloader)):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        out = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        loss = loss_fn(out, labels)

        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

        if batch_id % log_interval == 0:
            print(f"epoch {e+1} batch id {batch_id+1} train loss {(epoch_loss/(batch_id+1)):.3f}")

    model.eval()
    for batch_id, (input_ids, attention_mask, token_type_ids, labels) in enumerate(tqdm_notebook(valid_dataloader)):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        out = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        loss = loss_fn(out, labels)
        valid_loss += loss.item()
    valid_loss = valid_loss/len(valid_dataloader)
    print(f"epoch {e+1} batch id {batch_id+1} valid loss {(valid_loss):.3f}")

    if valid_loss < best_loss:
        best_loss = valid_loss
        best_epoch = e+1

        save_path = "./model.pt"
        torch.save({
            "epoch" : best_epoch,
            "model_state_dict" : model.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict(),
            "loss" : best_loss,
        }, save_path)

cuda:0


  0%|          | 0/941814 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 4.53 GiB already allocated; 0 bytes free; 4.78 GiB reserved in total by PyTorch)

In [17]:
try:
    checkpoint = torch.load("./model.pt")
    epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    loss = checkpoint['loss']
    print(f"Load checkpoint state. epoch is {epoch}, loss is {loss}")
except:
    print("No checkpoint state exits")
    epoch = 0
    loss = float('inf')

Load checkpoint state. epoch is 1, loss is 1.0987966125592208


In [18]:
print("input two sentence")
sent1 = input("sentence 1: ")
sent2 = input("sentence 2: ")

print(f"sent1: {sent1}")
print(f"sent1: {sent2}")

input two sentence
sent1: 아 빨리 자고싶다
sent1: 어우 졸려 이제 일해야지


In [19]:
#확인용
class NLIDataset2(Dataset):
  def __init__(self, sent1, sent2, tokenizer):
    self.tokenizer = tokenizer
    sentence1 = []
    sentence2 = []

    sentence1.append(sent1)
    sentence2.append(sent2)

    inputs = tokenizer(sentence1, sentence2, padding=True, truncation=True)
    self.input_ids = inputs['input_ids']
    self.attention_mask = inputs['attention_mask']
    self.token_type_ids = inputs['token_type_ids']

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return (self.input_ids[idx], self.attention_mask[idx], self.token_type_ids[idx])

In [20]:
def collate_fn2(batch):
    input_ids = [item[0] for item in batch]
    attention_mask = [item[1] for item in batch]
    token_type_ids = [item[2] for item in batch]
    return torch.LongTensor(input_ids), torch.LongTensor(attention_mask), torch.LongTensor(token_type_ids)

In [21]:
inference_ds = NLIDataset2(sent1, sent2, tokenizer)
batch_size = 1
infer_dataloader = torch.utils.data.DataLoader(inference_ds, batch_size=batch_size, collate_fn=collate_fn2)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [22]:
model.eval()
for batch_id, (input_ids, attention_mask, token_type_ids) in enumerate(infer_dataloader):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)

    out = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    outputs = out.squeeze().tolist()

print(sent1)
print(sent2)
print("contradiction:\t {:.5f}".format(outputs[0]))
print("entailment:\t {:.5f}".format(outputs[1]))
print("neutral:\t {:.5f}".format(outputs[2]))

아 빨리 자고싶다
어우 졸려 이제 일해야지
contradiction:	 0.33877
entailment:	 0.35578
neutral:	 0.30546


In [26]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/api', methods=['POST'])
def post_json():
    data = request.get_json()
    sent1 = data['answer']
    sent2 = "아 오늘 또 만났네"
    
    inference_ds = NLIDataset2(sent1, sent2, tokenizer)
    batch_size = 1
    infer_dataloader = torch.utils.data.DataLoader(inference_ds, batch_size=batch_size, collate_fn=collate_fn2)

    model.eval()
    for batch_id, (input_ids, attention_mask, token_type_ids) in enumerate(infer_dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)

    out = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    outputs = out.squeeze().tolist()

    print(sent1)
    print(sent2)
    print("contradiction:\t {:.5f}".format(outputs[0]))
    print("entailment:\t {:.5f}".format(outputs[1]))
    print("neutral:\t {:.5f}".format(outputs[2]))
    response_data = {
        'message' : outputs[0]
    }
    return jsonify(response_data)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Oct/2023 00:11:59] "POST /api HTTP/1.1" 200 -


안녕하세요 반갑습니다 
아 오늘 또 만났네
contradiction:	 0.33877
entailment:	 0.35578
neutral:	 0.30546
